In [19]:
import os
import sys
from datetime import datetime
sys.path.append(os.path.realpath('..'))

import pandas as pd
from utils.queries import query_contract_logs
from utils.contract import get_event_name, get_event_list_by_protocol, get_event_abi, decode_log
from utils.prices import get_prices
from utils.events import unwrap_decoded_events
from utils.wbtc import build_token_supply, build_balance_diff, get_balance, get_value_usd
import plotly.express as px
import plotly.io as pio
import plotly.graph_objects as go

pio.templates.default = "plotly_white"

In [20]:
protocol = 'weth'
start_date = '2017-12-11'
end_date = '2020-04-30'
limit = None
file_name = f'{protocol}.pickle'
address = '0xC02aaA39b223FE8D0A0e5C4F27eAD9083C756Cc2'
decimals = 8

In [21]:
events = get_event_list_by_protocol(protocol)

In [ ]:
query = query_contract_logs(address, start_date=start_date, end_date=end_date, limit=None)
df = pd.read_gbq(query)
df['name'] = df.apply(get_event_name, events=events, axis=1)
df['abi'] = df.apply(get_event_abi, events=events, axis=1)
df['decoded'] = df.apply(decode_log, events=events, axis=1)
df.to_pickle(file_name)
df = pd.read_pickle(file_name)

Downloading:  84%|████████▍ | 5492609/6526305 [1:02:01<03:29, 4929.71rows/s]

In [15]:
price = get_prices(["ETH"], start_date, end_date)

In [16]:
df.head()

,log_index,transaction_hash,transaction_index,address,data,topics,block_timestamp,block_number,block_hash,name,abi,decoded
0,49,0xa505f0db8c792e85c56efe20929229121ea4e47e37c7...,75,0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2,0xf0000000000000000000000000000000000000000000...,[0x8c5be1e5ebec7d5bd14f71427d1e84f3dd0314c0f7b...,2019-04-06 07:06:40+00:00,7513141,0x3721cd62c008433aaad2cadf6e6483cb4ee296e63b17...,Approval,"{'anonymous': False, 'inputs': [{'indexed': Tr...",{'src': '0x24496f44b0d3a3230e63c2289bd5e460127...
1,3,0xc6e069112f91c4e7fe8a2c149a5d19d370433a087d30...,3,0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2,0x00000000000000000000000000000000000000000000...,[0xddf252ad1be2c89b69c2b068fc378daa952ba7f163c...,2019-04-06 16:29:52+00:00,7515623,0xfc46ed165e3f42a68a35c1ce21bad94d7dc5447583e0...,Transfer,"{'anonymous': False, 'inputs': [{'indexed': Tr...",{'src': '0xad9eb619ce1033cc710d9f9806a2330f858...
2,18,0x2b43c437e8f49d1d9d9789ca2870f8af4eead60978b0...,56,0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2,0x00000000000000000000000000000000000000000000...,[0xddf252ad1be2c89b69c2b068fc378daa952ba7f163c...,2019-04-06 16:16:36+00:00,7515574,0x84729305588fe836838c69fc4c47edd27bcc2ed2e811...,Transfer,"{'anonymous': False, 'inputs': [{'indexed': Tr...",{'src': '0x2af2aa9b7712d7485f63fef5e03cd8f68f3...
3,19,0x2b43c437e8f49d1d9d9789ca2870f8af4eead60978b0...,56,0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2,0x00000000000000000000000000000000000000000000...,[0x8c5be1e5ebec7d5bd14f71427d1e84f3dd0314c0f7b...,2019-04-06 16:16:36+00:00,7515574,0x84729305588fe836838c69fc4c47edd27bcc2ed2e811...,Approval,"{'anonymous': False, 'inputs': [{'indexed': Tr...",{'src': '0x9d997393d31189acea8b984bfde003f94c8...
4,24,0x2b43c437e8f49d1d9d9789ca2870f8af4eead60978b0...,56,0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2,0x00000000000000000000000000000000000000000000...,[0xddf252ad1be2c89b69c2b068fc378daa952ba7f163c...,2019-04-06 16:16:36+00:00,7515574,0x84729305588fe836838c69fc4c47edd27bcc2ed2e811...,Transfer,"{'anonymous': False, 'inputs': [{'indexed': Tr...",{'src': '0x9d997393d31189acea8b984bfde003f94c8...


# Charts


### Usage of events

In [17]:
fig = px.pie(df['name'].value_counts().reset_index(), values='name', names='index')
fig.show()

### Total supply over time

In [9]:
total_supply = build_token_supply(df, decimals)

KeyError: 'amount'

In [10]:
fig = px.line(total_supply.reset_index(), x="block_timestamp", y="value")
fig.update_yaxes(title_text='Total supply')
fig.update_xaxes(title_text='Date')
fig.show()

NameError: name 'total_supply' is not defined

### Balance of key holders over time

In [ ]:
transfer = unwrap_decoded_events(df, 'Transfer')

In [ ]:
balance_diff = build_balance_diff(transfer, decimals)
temp_historical_balance = []
for i in range(0, 17):
    date = pd.to_datetime('2019-01-01', utc=True) + pd.DateOffset(months=i)
    temp_historical_balance.append(get_balance(balance_diff, date))
historical_balance = pd.concat(temp_historical_balance)

In [ ]:
labels = pd.read_csv('../utils/labels.csv')

In [ ]:
label_balance = historical_balance.set_index('address').join(labels.set_index('address'), rsuffix='_label').reset_index().fillna('others')

In [ ]:
grouped_label_balance = label_balance.groupby(['date', 'name'])['value'].sum().reset_index()
fig = px.area(grouped_label_balance, x="date", y="value", color="name")
fig.update_yaxes(title_text='Balance')
fig.update_xaxes(title_text='Date')
fig.show()

### Volume

In [ ]:
transfer = get_value_usd(transfer, price, decimals, value_column='value')

In [ ]:
volume = transfer.set_index('date').resample('M')['value_usd'].sum().reset_index()
volume['date'] = volume['date'] - pd.DateOffset(months=1)
fig = px.area(volume, x="date", y="value_usd")
fig.update_yaxes(title_text='Volume in USD')
fig.update_xaxes(title_text='Date')
fig.show()

### Unique addresses

In [ ]:
unique = balance_diff.set_index('block_timestamp').resample('M')['address'].nunique().reset_index()
unique['block_timestamp'] = unique['block_timestamp'] - pd.DateOffset(months=1)
fig = px.area(unique, x="block_timestamp", y="address")
fig.update_yaxes(title_text='Unique addresses')
fig.update_xaxes(title_text='Date')
fig.show()

In [ ]:
unique.head()